### Use sctransform conda environment

In [ ]:
library(Matrix)
library(sctransform)
library(reticulate)
np <- import("numpy")

In [2]:
data_dir = "../data/scRNAseq_YangLi/"

In [3]:
data = readMM(paste(data_dir, "bcg0712_complete.QC2.mtx", sep="/"))
var_names = rownames(read.csv(paste(data_dir, "bcg0712_complete.QC2.var.csv", sep="/"), row.names=1))
obs_names = rownames(read.csv(paste(data_dir, "bcg0712_complete.QC2.obs.csv", sep="/"), row.names=1))
data = t(data)
rownames(data) = var_names
colnames(data) = obs_names
data = as(data, "dgCMatrix")
str(data)

Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:307491164] 15 25 28 39 43 46 56 59 72 76 ...
  ..@ p       : int [1:181447] 0 3268 6150 8948 10780 13469 16644 18098 20176 23354 ...
  ..@ Dim     : int [1:2] 20856 181446
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:20856] "AL627309.1" "AL627309.5" "AL627309.4" "LINC01409" ...
  .. ..$ : chr [1:181446] "AAACCCAAGTCTAGAA-1_1_1" "AAACCCACAATTTCCT-1_1_1" "AAACGAACAGCGTTTA-1_1_1" "AAACGAAGTCGCATGC-1_1_1" ...
  ..@ x       : num [1:307491164] 1 1 1 1 1 5 3 2 3 1 ...
  ..@ factors : list()


In [4]:
options(future.globals.maxSize=2048*1024^2)
vst_out = sctransform::vst(data, latent_var=c('log_umi'), method='glmGamPoi',
                           return_corrected_umi=TRUE, return_gene_attr=TRUE, return_cell_attr=TRUE)
saveRDS(vst_out, file=paste(data_dir, "bcg0712_complete.QC2.SCT.rds", sep="/"))

Calculating cell attributes from input UMI matrix: log_umi

Variance stabilizing transformation of count matrix of size 20856 by 181446

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 2000 genes, 181446 cells



  |======================================================================| 100%


Found 69 outliers - those will be ignored in fitting/regularization step


Second step: Get residuals using fitted parameters for 20856 genes



  |======================================================================| 100%


Computing corrected count matrix for 20856 genes



  |======================================================================| 100%


Calculating gene attributes

Wall clock passed: Time difference of 17.52226 mins



In [5]:
# vst_out = readRDS(paste(data_dir, "bcg0712_complete.QC2.SCT.rds", sep="/"))

In [6]:
str(vst_out)

List of 14
 $ y                    : num [1:20856, 1:181446] -0.0584 -0.1096 -0.011 -0.2257 -0.2901 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:20856] "AL627309.1" "AL627309.5" "AL627309.4" "LINC01409" ...
  .. ..$ : chr [1:181446] "AAACCCAAGTCTAGAA-1_1_1" "AAACCCACAATTTCCT-1_1_1" "AAACGAACAGCGTTTA-1_1_1" "AAACGAAGTCGCATGC-1_1_1" ...
 $ model_str            : chr "y ~ log_umi"
 $ model_pars           : num [1:2000, 1:3] 2.797 1.027 0.189 0.396 2.242 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:2000] "MALAT1" "FTH1" "AC122129.1" "ZNF213-AS1" ...
  .. ..$ : chr [1:3] "theta" "(Intercept)" "log_umi"
 $ model_pars_outliers  : logi [1:2000] FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ model_pars_fit       : num [1:20856, 1:3] 0.02783 0.07774 0.00476 0.24792 0.36346 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:20856] "AL627309.1" "AL627309.5" "AL627309.4" "LINC01409" ...
  .. ..$ : chr [1:3] "theta" "(Intercept)" "log_umi"
  ..- attr(*, "outliers")= logi

In [7]:
write.table(rownames(vst_out$y), file=paste(data_dir, "bcg0712_complete.QC2.SCT.var.csv", sep="/"), row.names=FALSE, col.names=FALSE)
write.table(colnames(vst_out$y), file=paste(data_dir, "bcg0712_complete.QC2.SCT.obs.csv", sep="/"), row.names=FALSE, col.names=FALSE)
np$save(paste(data_dir, "bcg0712_complete.QC2.SCT_residuals.npy", sep="/"), vst_out$y)
write.table(vst_out$gene_attr$residual_variance, file=gzfile(paste(data_dir, "bcg0712_complete.QC2.SCT_variance.csv.gz", sep="/")), sep=',', row.names=TRUE, col.names=FALSE)
writeMM(vst_out$umi_corrected, file=paste(data_dir, "bcg0712_complete.QC2.SCT_corr_counts.mtx", sep="/"))

NULL

In [ ]:
# write.table(vst_out$y, file=gzfile(paste(data_dir, "bcg0712_complete.QC2.SCT_residuals.csv.gz", sep="/")), sep=',', row.names=TRUE, col.names=FALSE)